# Quantum Circuit Simulator
### James Saslow
### 12/9/2023

In [2]:
# Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
class Wavefunction:
    '''
    Class Description:

    This class takes in a numpy array to create an object Wavefunction
    A series of gate based operations can be applied to modify the Wavefunction object

    Gate Based operation functions:
        NOT:      Wavefunction.x(targets)
        CNOT      Wavefunction.cx(target, control)
        Hadamard: Wavefunction.h(targets)
        SWAP:     Wavefunction.swap(qubit a, qubit b)
        Phase:    Wavefunction.p()
        Toffoli:  Wavefunction.ccx([control a, control b], target)
        ...

    Data visualization functions:
        Wavefunction.get()
        Wavefunction.show()
        Wavefunction.hist()

    '''
    #=================== Defining Quantum Gates =========================
    # Identity
    I = np.array([[1,0],
                  [0,1]])

    # Pauli X
    X = np.array([[0,1],
                  [1,0]])

    # Pauli Y
    Y = np.array([[0,-1j],
                  [1j,0]], dtype = complex)

    # Pauli Z
    Z = np.array([[1,0],
                  [0,-1]])

    # Hadamard
    H = (X+Z)/np.sqrt(2)

    # SWAP
    SWAP = np.array([[1,0,0,0],
                     [0,0,1,0],
                     [0,1,0,0],
                     [0,0,0,1]])

    # Control - Not
    CX = np.array([[1,0,0,0],
                   [0,1,0,0],
                   [0,0,0,1],
                   [0,0,1,0]])

    # Toffoli Gate
    CCX = np.array([[1,0,0,0,0,0,0,0],
                   [0,1,0,0,0,0,0,0],
                   [0,0,1,0,0,0,0,0],
                   [0,0,0,1,0,0,0,0],
                   [0,0,0,0,1,0,0,0],
                   [0,0,0,0,0,1,0,0],
                   [0,0,0,0,0,0,0,1],
                   [0,0,0,0,0,0,1,0]])

    #========================================================================================
    #=================== Data Visualization Functions & Class Functions  ====================

    def __init__(self, psi):
        self.psi = psi

    def get(self):
        '''
        Function Description
        '''

        psi = self.psi
        return psi

    def show(self, **kwargs):
        '''
        Function Description

        **kwargs:
        base = prints kets in base 2 or base 10
        dec = rounding precision
        polar = prints phases in polar form (Need to implement later)
        '''

        psi = self.psi
        num_bases  = len(psi)
        num_qubits = int(np.log2(num_bases))

        bases = []


        base = 2 # Defaults to base 2 ket labels
        if 'base' in kwargs:
            base = int( kwargs['base'] )

        if base == 10:
            # Generating ket labels in Base 10
            for i in range(num_bases):
                bases.append('|' + str(i) + '>')
        else:
            # Generating Ket Labels in Base 2
            for i in range(num_bases):
                bin_num = bin(i)[2:]
                bin_num = '0'*(num_qubits - len(bin_num)) + bin_num
                bin_num = '|'+bin_num + '>'
                bases.append(bin_num)

        # Rounding psi to 'dec' points of precision (defaults at dec = 5)
        dec = 5
        if 'dec' in kwargs:
            dec = int( kwargs['dec'] )

        polar_bool = kwargs.get('polar', False)
        printed_psi = []
        if polar_bool == True:
            # Storing wavefunction in polar format
            for i in range(num_bases):
                r = np.round(np.abs(psi[i]),dec )
                theta = np.round(np.angle(psi[i]),dec)
                if r == 0:
                    printed_psi.append('0')
                else:
                    printed_psi.append(str(r) + ' exp(1j*' + str(theta) + ')')
        else:
            # Storing wavefunction in cartesian format
            rounded_psi = np.round(psi,dec)
            for i in range(num_bases):
                if np.abs(rounded_psi[i]) == 0:
                    printed_psi.append('0')
                else:
                    printed_psi.append(str(rounded_psi[i]))


        # Determining number of spaces to have alligned formatting
        char_arr = []
        for i in range(num_bases):
            char_arr.append(len(printed_psi[i]))
        max_char = max(char_arr)
        spaces = max_char - np.array(char_arr)


        # Printing the formatted wavefunction
        zeros_bool = kwargs.get('zeros', False)
        for i in range(num_bases):
            if zeros_bool == False:
                if printed_psi[i] == '0':
                    continue # Only printing non-zero states if zeros_bool = False
                else:
                    print(printed_psi[i], ' '*spaces[i], bases[i])
            else:
                print(printed_psi[i], ' '*spaces[i], bases[i]) # Printing every state if zeros_bool = True
        print(' ')

    #========================================================================================
    #=========================== General Gate-Based Operations  =============================

    # In general for any single qubit gate
    def single_qubit_gate(self,gate,target_qubit):
        '''
        Function Description

        '''
        psi = self.psi
        I = self.I

        num_bases = len(psi)                 # Number of basis vectors in psi
        num_qubits = int(np.log2(num_bases)) # Number of qubits in psi

        # If set to target all qubits, target_qubit array contains all qubits
        if target_qubit == 'all':
            target_qubit = np.arange(num_qubits)

        # Changing int data types to lists
        if type(target_qubit) == int:
            target_qubit = [target_qubit]

        # Exception Handling
        for i in target_qubit:
            if i>num_qubits-1:
                raise ValueError('Targeted Qubit not in Hilbert Space')

        # Initializing 0th M Matrix
        M = I
        for i in target_qubit:
            if i == 0:
                M = gate

        # Tensoring all other M Matrices
        for i in range(1,num_qubits):
            if i in target_qubit:
                M = np.kron(M,gate)
            else:
                M = np.kron(M,I)

        # Returning the gate acting on psi
        self.psi = np.matmul(M,psi)



    # In general for any 2 qubit gate
    def two_qubit_gate(self, gate, control_qubit, target_qubit):
        '''
        Function Description
        '''
        # Control and Target must be neighboring qubits
        psi = self.psi
        I = self.I
        SWAP = self.SWAP

        num_bases = len(psi)
        num_qubits = int(np.log2(num_bases))

        if (target_qubit - control_qubit) == 1:
            pass
        elif (target_qubit - control_qubit) == -1:
            gate = SWAP @ gate @ SWAP # Reversing target and control for asymmetric gates
        else:
            raise ValueError('Target and Control must be neighboring qubits')

        # Choosing where to start for loop if the first gate is a two qubit gate or identity
        if 0 in [control_qubit, target_qubit]:
            M = gate
            i_init = 2
        else:
            M = I
            i_init = 1

        skip = False
        for i in range(i_init, num_qubits):

            # Skips an iteration if a two qubit gate is already applied
            if skip == True:
                skip = False
                continue

            if i in [control_qubit, target_qubit]:
                M = np.kron(M,gate)
                skip = True
            else:
                M = np.kron(M,I)

        self.psi = np.matmul(M, psi)


    def three_qubit_gate(self, gate, control1_qubit, control2_qubit, target_qubit):
        psi = self.psi
        I = self.I
        SWAP = self.SWAP

        num_bases = len(psi)
        num_qubits = int(np.log2(num_bases))

        control = np.array([control1_qubit, control2_qubit])
        diff = target_qubit - control
        sum_diff = sum(diff)

        if sum_diff == 3: # CCT configuration
            pass
        if sum_diff == 0: # CTC configuration
            A = np.kron(I,SWAP)
            gate = (A @ gate @ A)
        if sum_diff == -3: # TCC configuration
            A1 = np.kron(I,SWAP)
            A2 = np.kron(SWAP,I)
            gate =  (A2 @ A1 @ gate @ A1 @ A2)

        total = [control1_qubit, control2_qubit, target_qubit]
        if 0 in total: # Checking if we're starting out with a gate
            M = gate
            i_init = 3
        else:
            M = I
            i_init = 1

        skip = 0
        for i in range(i_init, num_qubits):
            if skip >0:
                skip = skip -1
                continue

            if i in total:
                M = np.kron(M, gate)
                skip = 2 # Skipping 2 iterations if a 3 qubit gate is applied
            else:
                M = np.kron(M,I)

        self.psi = np.matmul(M,psi)


    #======================================================================================
    # ======================== Specific Gate Based Functions ==============================

    # Hadamard Gate
    def h(self, target_qubit):
        H = self.H
        psi = self.psi
        self.single_qubit_gate(H, target_qubit)

    # X Gate
    def x(self, target_qubit):
        X = self.X
        psi = self.psi
        self.single_qubit_gate(X, target_qubit)

    # Y Gate
    def y(self, target_qubit):
        Y = self.Y
        psi = self.psi
        self.single_qubit_gate(Y, target_qubit)

    # Z Gate
    def z(self, target_qubit):
        Z = self.Z
        psi = self.psi
        self.single_qubit_gate(Z,target_qubit)

    # Phase Gate
    def p(self, target_qubit, theta):
        psi = self.psi
        P = np.array([[1,0],
                      [0,np.exp(1j*theta)]])
        self.single_qubit_gate(P,target_qubit)

    # Control Not Gate
    def cx(self, control_qubit, target_qubit):
        CX = self.CX
        psi = self.psi
        self.two_qubit_gate(CX, control_qubit, target_qubit)

    # SWAP Gate
    def swap(self, control_qubit, target_qubit):
        SWAP = self.SWAP
        psi = self.psi
        self.two_qubit_gate(SWAP, control_qubit, target_qubit)

    # Toffoli Gate
    def ccx(self, control_qubits, target_qubit):
        CCX = self.CCX
        psi = self.psi
        control1_qubit, control2_qubit = control_qubits
        self.three_qubit_gate(CCX, control1_qubit, control2_qubit, target_qubit)



# Quantum Circuit Tutorial

A brief overview of the functionality of the Wavefunction class

In [4]:
print('________________Quantum Circuit Simulator Tutorial________________')

# 1) Define a wavefunction as a numpy array
psi = np.array([1,-1,0,1])/np.sqrt(3)

# 2) Create Wavefunction class

wf = Wavefunction(psi)
print('Wavefunction class: ', wf)
print(' ')

# 3) Data Visualization and data retrevial

print('Wavefunction.get() returns psi as a numpy array')
print(wf.get())
print(' ')

print('Wavefunction.show() prints the wavefunction in bra-ket notation')
wf.show()

print('Wavefunction.show(dec = dec) sets the precision limit to dec')
wf.show(dec = 10)

print('Wavefunction.show(zeros = True) displays states with 0 amplitude')
wf.show(zeros = True)

print('Wavefunction.show(polar = True) displays wavefunction in a polar format')
wf.show(polar = True)

# 4) Gate based operations are shown below
print('Gate based operation functions are shown below')

________________Quantum Circuit Simulator Tutorial________________
Wavefunction class:  <__main__.Wavefunction object at 0x000001AB9197A348>
 
Wavefunction.get() returns psi as a numpy array
[ 0.57735027 -0.57735027  0.          0.57735027]
 
Wavefunction.show() prints the wavefunction in bra-ket notation
0.57735   |00>
-0.57735  |01>
0.57735   |11>
 
Wavefunction.show(dec = dec) sets the precision limit to dec
0.5773502692   |00>
-0.5773502692  |01>
0.5773502692   |11>
 
Wavefunction.show(zeros = True) displays states with 0 amplitude
0.57735   |00>
-0.57735  |01>
0         |10>
0.57735   |11>
 
Wavefunction.show(polar = True) displays wavefunction in a polar format
0.57735 exp(1j*0.0)      |00>
0.57735 exp(1j*3.14159)  |01>
0.57735 exp(1j*0.0)      |11>
 
Gate based operation functions are shown below


# Implementing Grover's Algorithm

In [57]:
# ========== Initializing Wavefunction ==========
num_qubits = 2
psi = np.zeros(2**num_qubits)
psi[0] = 1

wf = Wavefunction(psi) # Creating Wavefunction class

print('____Quantum Register Initialization____')
print('|\u03C8> = ')
wf.show() # Printing wavefunction

# ========== Transforming to Equal Superposition State ==========
wf.h('all')

print('____Equal Superposition State____')
print('|\u03C8> = ')
wf.show() # Printing wavefunction

# ========== Choosing which item we want to search ==========

target = [1,0] # User can choose which binary bit string to search for

# Applying a not gate on the associated qubit where the target = '0'
for i in range(len(target)):
    if target[i] == 0:
        wf.x(i)
    else:
        pass

print('____Choosing Item to Search (Preprocessing)____')
print('Searching for:', target)


print('|\u03C8> = ')
wf.show() # Printing wavefunction


# ========== Applying Quantum Oracle ==========

# Applying the Circuit Equivalent to a CCZ
wf.h(1)
wf.cx(0,1)
wf.h(1)

print('____Applying Quantum Oracle____')
print('|\u03C8> = ')
wf.show()


# # ========== Applying Item search choice after oracle too ==========

for i in range(len(target)):
    if target[i] == 0:
        wf.x(i)
    else:
        pass

print('____Choosing Item to Search (Post-Processing)____')

print('|\u03C8> = ')
wf.show() # Printing wavefunction

# ========== Applying Diffusion Operator ============

wf.h('all')
wf.x('all')

wf.h(1)
wf.cx(0,1)
wf.h(1)

wf.x('all')
wf.h('all')

print('____Applying Diffusion Operator____')
wf.show()

____Quantum Register Initialization____
|ψ> = 
1.0  |00>
 
____Equal Superposition State____
|ψ> = 
0.5  |00>
0.5  |01>
0.5  |10>
0.5  |11>
 
____Choosing Item to Search (Preprocessing)____
Searching for: [1, 0]
|ψ> = 
0.5  |00>
0.5  |01>
0.5  |10>
0.5  |11>
 
____Applying Quantum Oracle____
|ψ> = 
0.5   |00>
0.5   |01>
0.5   |10>
-0.5  |11>
 
____Choosing Item to Search (Post-Processing)____
|ψ> = 
0.5   |00>
0.5   |01>
-0.5  |10>
0.5   |11>
 
____Applying Diffusion Operator____
-1.0  |10>
 
